In [1]:
import numpy as np
import pandas as pd
import collections
import funcy
import os
import re
import itertools
from dev import (LOCAL_DB, DATA_DIR, DROPBOX_DIR,
                     REPO_STATS_FNAME, COMMIT_STATS_FNAME, DEV_COLLAB_FNAME, DEV_CHANGES_FNAME,
                 DEV_CONTR_BY_FILE_FNAME,
                         FILE_DEV_CUM_STATS_FNAME, FILE_ADJ_MATRIX_FNAME, 
                 FILE_STATS_FNAME, FILE_DEV_CUM_STATS_FNAME, RAW_FNAME)
    
from models import Commit, Change, Developer, Diff

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
EXPORT_TO_CSV = True

In [3]:
FIGURE_DIR = os.path.join(DATA_DIR, 'figures')

In [4]:
from helpers import connect_to_db

In [5]:
session, engine = connect_to_db(LOCAL_DB)

/Users/Lraymond/Python/sourcecred_research/sourcecred_research3.6/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [6]:
pd_dict = {
    'filename_new': [],
    'filename_old': [],
    'commit_id': None,
    'additions': 0,
    'deletions': 0,
    'is_deletion': False,
    'is_rename': False,
    'is_new': False,
    'commit_timestamp': None,
    'commit_subject': '',
    'commit_body': '',
    'author_id': None,
    'commit_id': None,
}

In [7]:
# calculate the number of unique filenames
# filename_tups = session.query().with_entities(
#     Diff.filename_new, Diff.filename_old, Diff.commit_id, 
#         Diff.additions, Diff.deletions, Diff.is_deletion, Diff.is_rename, 
#             Diff.is_new).all()

filename_tups = session.query(Diff).all()

In [8]:
diff_df = pd.DataFrame(list(map(lambda x: x.__dict__, filename_tups)))
diff_df.rename(columns={'id': 'diff_id'}, inplace=True)
diff_df2 = diff_df.drop_duplicates(subset=['filename_new', 'filename_old', 'commit_id'])

In [9]:
print(diff_df.shape)
print(diff_df2.shape)

(10762, 12)
(4739, 12)


In [10]:
del diff_df2['_sa_instance_state']

In [11]:
commits_df = pd.DataFrame(list(map(lambda x: x.__dict__, session.query(Commit).all())))
commits_df.rename(columns={'id': 'commit_id'}, inplace=True)

In [12]:
del commits_df['_sa_instance_state']
del commits_df['raw_text']

In [13]:
df = pd.merge(left=diff_df2, right=commits_df, on='commit_id')
print(df.shape)

(4739, 20)


In [14]:
# merge in author information
author_df = pd.DataFrame(list(map(lambda x: x.__dict__, session.query(Developer).all())))

In [15]:
del author_df['_sa_instance_state']

In [16]:
# merge in author
auth2 = author_df.rename(columns={'email': 'author_email', 'id': 'author_id'})
df2 = df.merge(right=auth2[['author_email', 'author_id']], on='author_id')
auth3 = author_df.rename(columns={'email': 'commiter_email', 'id': 'commiter_id'})
df3 = df2.merge(right=auth3[['commiter_email', 'commiter_id']], on='commiter_id')


In [17]:
del auth2, auth3

In [18]:
# Here I should create something that looks for file renames and maps old name to new name and creates a new column (stable filename)
print('Number of renames', df3.loc[df3.is_rename==True].shape)
print('Total shape ', df3.shape)


Number of renames (98, 22)
Total shape  (4739, 22)


In [19]:
# for the ones with blank files names, search for "rename from " to "rename to"
def find_renames(raw_diff_text):
    m = re.search(r'rename from (.+\.\w{1,10})(\|\|\|\||\|\|\|)rename to (.+\.\w{1,10})', raw_diff_text)
    if m:
        res = m.groups()
        if len(res) == 3:
            return res[0], res[2]
        return res
    return None

In [20]:
missing_filenames = ((df3.is_rename==True) & (df3.filename_new=='') & (df3.filename_old==''))

In [21]:
replacement_names = df3['raw_diff'].apply(find_renames)

In [22]:
df3.loc[missing_filenames, 'filename_old'] = replacement_names[missing_filenames].apply(lambda x: x[0])
df3.loc[missing_filenames, 'filename_new'] = replacement_names[missing_filenames].apply(lambda x: x[1])
df3[['is_deletion', 'is_new', 'is_rename']] = df3[['is_deletion', 'is_new', 'is_rename']].astype(int)

In [23]:
# test to see if fillin gin the renames worked
# df3.loc[missing_filenames, ['filename_new', 'filename_old', 'raw_diff']]

In [24]:
# create a set of tups of all filename matches (old, new)
filename_tups = df3[['timestamp', 'filename_old', 'filename_new']].sort_values('timestamp').values

In [25]:
canonical_name_to_other_names_dict = dict()
# dict that goes from canonical names to other file names

other_name_to_canonical_name_dict = dict()
# dict that does from file names to canonical name

for time_val, old_name, new_name in filename_tups:
    if old_name == '/dev/null':
        # this is a file creation
        if new_name not in canonical_name_to_other_names_dict:
            canonical_name_to_other_names_dict[new_name] = [new_name]
            other_name_to_canonical_name_dict[new_name] = new_name
            
    elif new_name == '/dev/null':
        # this is a file deletion
        # need to check that it exists in the dict because not totally capturing everything perfectly
        if old_name not in other_name_to_canonical_name_dict:
            # we haven't done anything with this filename yet
            canonical_name_to_other_names_dict[old_name] = [old_name]
            other_name_to_canonical_name_dict[old_name] = old_name
            
    elif new_name==old_name:
        # new name is the same as old_name
         if new_name not in other_name_to_canonical_name_dict:
            # we haven't done anything with this filename yet
            canonical_name_to_other_names_dict[new_name] = [new_name]
            other_name_to_canonical_name_dict[new_name] = new_name
    elif new_name != old_name:
        # and neither new name nor old name is dev/null
        if old_name in other_name_to_canonical_name_dict:
            can_name = other_name_to_canonical_name_dict[old_name]
            if can_name not in canonical_name_to_other_names_dict:
                can_name = other_name_to_canonical_name_dict[can_name]
            canonical_name_to_other_names_dict[can_name].append(new_name)
            other_name_to_canonical_name_dict[new_name] = old_name
        else:
            # this is a weird pair
            print(old_name, new_name) 
            other_name_to_canonical_name_dict[old_name] = old_name
            other_name_to_canonical_name_dict[new_name] = old_name
            canonical_name_to_other_names_dict[old_name] = [new_name]
            

tests/test-repo/blocks/1220c0fc/1220c0fc6b49543d7bf04e83d2a5a7cbe72a83e80f9c7bca1abcaa42298a57a33ff5.data tests/repo-example/blocks/1220c0fc/1220c0fc6b49543d7bf04e83d2a5a7cbe72a83e80f9c7bca1abcaa42298a57a33ff5.data
tests/repo-example/blocks/1220933b/1220933b41d37fd4508cdff45930dff56baef91c7dc345e73d049ab570abe10dfbb9.data test/go-ipfs-repo/blocks/1220933b/1220933b41d37fd4508cdff45930dff56baef91c7dc345e73d049ab570abe10dfbb9.data
test/http-api/test-bitswap.js test/http-api/ipfs-api/test-bitswap.js
test/go-ipfs-repo/blocks/1220b0cb/1220b0cba7371f11461f77081b947d837cc9a3b80e182ac123bbd427563e8b167c96.data test/go-ipfs-repo/blocks/CIQLB/CIQLBS5HG4PRCRQ7O4EBXFD5QN6MTI5YBYMCVQJDXPKCOVR6RMLHZFQ.data
test/http-api/spec/test-pubsub.js test/http-api/spec/pubsub.js
test/http-api/spec/test-config.js test/http-api/spec/config.js
test/go-ipfs-repo/blocks/CIQER/CIQERMRAAFXUAUOX3V2DCW7R77FRIVHQ3V5OIPPS3XQBX34KRPNOIRQ.data test/test-data/go-ipfs-repo/blocks/CIQER/CIQERMRAAFXUAUOX3V2DCW7R77FRIVHQ3V5OIPPS

In [26]:
# test canonical dictionary
multiple_renames = funcy.select_values(lambda x: len(x) > 1, canonical_name_to_other_names_dict)
for old_name, new_name in df3.loc[df3.is_rename==True, ['filename_old', 'filename_new']].values:
    if old_name in canonical_name_to_other_names_dict:
        if new_name not in canonical_name_to_other_names_dict[old_name]:
            res = funcy.select_values(lambda x: new_name in x, multiple_renames)
            if len(res) > 1:
                print(res)
            else:
                print('Nothing found')
                print(old_name, new_name)
            

Nothing found
test/core-tests/test-init.js test/core/both/test-init.js
Nothing found
examples/basics/index.js /dev/null


In [27]:
# find unique filenames, then loop through all unique filenames, if part of a rename, 
# append to the dict associated with that filename. if entries in that dictionary, if a rename to one of those, associate with that

canonical_name_to_other_names_dict[
    'test/core-tests/test-init.js'] = ['test/core-tests/test-init.js', 'test/core/both/test-init.js']

In [28]:
# create a skeleton of all names in x axis mapped to canonical name (invert dictionary)
names_final = dict()

for k, v in canonical_name_to_other_names_dict.items():
    for i in v:
        names_final[i] = k
            

In [29]:
# map these names onto the data set
def get_canonical_name(new_name, old_name, name_dict):
    if new_name == '/dev/null':
        return name_dict[old_name]
    elif old_name == '/dev/null':
        return name_dict[new_name]
    else:
        if old_name in name_dict:
            return name_dict[old_name]
        elif new_name in name_dict:
            return name_dict[new_name]
        print('Bad Case ', new_name, old_name)
        return new_name
    

In [30]:
df3['canonical_name'] = df3.apply(
    lambda x: get_canonical_name(x['filename_new'], x['filename_old'], names_final), axis=1)

In [31]:
from gitlog_parser import is_raw_diff

def code_complexity(text_str):
    return len(text_str) - len(text_str.lstrip())
    
def analyze_diff(text_str):
    lines = text_str.split('||||')
    if len(lines)==1:
        lines = text_str.split('|||')
    complexity = []
    for line in lines:
        if line.startswith('+'):
            complexity.append(code_complexity(line[1:]))
    return complexity

    

In [32]:
df3['code_complexity'] = df3.raw_diff.apply(analyze_diff)
df3['code_complexity_max'] = df3.code_complexity.apply(lambda x: 0 if len(x)==0 else np.max(x))
df3['code_complexity_min'] = df3.code_complexity.apply(lambda x: 0 if len(x)==0 else np.min(x))
df3['code_complexity_mean'] = df3.code_complexity.apply(lambda x: 0 if len(x)==0 else np.mean(x))
df3['code_complexity_median'] = df3.code_complexity.apply(lambda x: 0 if len(x)==0 else np.median(x))
df3['net_change'] = df3.additions - df3.deletions

In [33]:
if EXPORT_TO_CSV:
    df3.to_csv(os.path.join(DROPBOX_DIR, RAW_FNAME))

# Repo Level Stats

* total lines of code added
* total lines of code deleted
* total number of files add, deleted, renamed
* total number of files
* total net code

In [ ]:
time_stats = df3.sort_values('timestamp').groupby('timestamp').agg({
    'additions': np.sum, 'deletions': np.sum, 'net_change': np.sum,
    'filename_old': 'nunique', 'diff_id': 'count', 
    'is_rename': np.sum, 'is_deletion': np.sum, 'is_new': np.sum}).rename(columns={
        'additions': 'total_lines_added', 
        'deletions': 'total_lines_deleted',
        'net_change': 'total_lines_code',
        'filename_old': 'total_num_unique_files',
        'diff_id': 'total_num_edit_locations',
         'is_rename': 'total_num_renames',
        'is_deletion': 'total_num_deletions',
        'is_new': 'total_num_new_files'})

In [ ]:
# this is a cumulative sum along time axis
cum_repo_stats = time_stats.cumsum()

In [ ]:
cum_repo_stats.columns = ['cum_{}'.format(c) for c in cum_repo_stats.columns]

In [ ]:
repo_level = cum_repo_stats.merge(time_stats, left_index=True, right_index=True, how='outer')

In [ ]:
print(time_stats.shape)
print(cum_repo_stats.shape)
print(repo_level.shape)


In [ ]:
if EXPORT_TO_CSV:
    repo_level.to_csv(os.path.join(DROPBOX_DIR, REPO_STATS_FNAME))

### Commit level code info
* calculate amount of net code
* plot net code, additions, deletions in each change
* calculate number of file changed in each commit 
* for each commit, also want total number of change in each commit
* for each commit, try to get average change size


### file level
* cumulative number of changes
* cumulative number of devs who work on them
* code churn - number of changes relative to total changes
* average code change per change to each file

### file pairs
* create count of all files changed together and show which are most commonly changed together


### Developer
* number of changes per developer
* total lines added/deleted per developer
* number of created/deleted/renamed files per dev
* create pairs of devs who work together



### Oustanding To Dos
* need to fix rename fail in database pipelines
* need to add files to ignore
* why are tests failing after that change?
* code complexity count per diff
* are cumulative things being aggregated properly?
* How to combine all the data into a useful format/figure out what is important?
* What about adding in text from comments on each commit?
* Are we missing any files?
* What about file dependence?


# Questions
* how to aggregate code complexity?
* What is important
* why is net code < 0 
* what are the blank filenames?

### TO DO
I need to deal with the missing filenames - this is an example of a bigger issue, but only 164 at the moment
so will continue writing code to generate graphs

Also need to fix duplication issue in the database for Diffs

# Commit level stats
    1) number of files changed per commit

    2) number of edit locations per commit

    3) average code complexity of each diff (note not exactly sure how to aggregate this)
    
    4) Total number of lines changed
    
    5) Committer/author same person

In [ ]:
adds = df3.sort_values(['commit_id', 'timestamp']).groupby('commit_id').agg({
    'additions': np.sum, 'deletions': np.sum, 'net_change': np.mean,
     'timestamp': np.min, 'code_complexity_median': np.mean, 
    'filename_old': 'nunique', 'diff_id': 'count', 
    'is_rename': np.sum, 'is_deletion': np.sum, 'is_new': np.sum})

In [ ]:
commit_stats = adds.rename(columns={
    'additions': 'total_additions', 
    'deletions': 'total_deletions', 'net_change': 'average_net_change',
    'code_complexity_median':'mean_of_code_complexity_median', 
    'filename_old': 'num_unique_files_changed', 'diff_id': 'num_edit_locations',
     'is_rename': 'num_renames', 'is_deletion': 'num_deletions',
        'is_new': 'num_new_files'})

In [ ]:
devs = df3.sort_values('commit_id').drop_duplicates('commit_id')[[
    'commit_id', 'author_id', 'commiter_id', 'author_email', 'commiter_email', 'commit_body',
    'git_hash', 'repo_id', 'sha1', 'subject']]
commit_stats2 = commit_stats.merge(devs, left_index=True, right_on='commit_id', how='left')

In [ ]:
# Count where developer and ocmmiter are different
commit_stats2['diff_dev_commit'] = commit_stats2.apply(
    lambda x: 0 if x['author_email']==x['commiter_email'] else 1, axis=1)

In [ ]:
if EXPORT_TO_CSV:
    commit_stats2.to_csv(os.path.join(DROPBOX_DIR, COMMIT_STATS_FNAME))

# Developer Level Stats

    * date of first engagement with project by author/commiter
    * type of first engagement
    * date of last time of engagement with project
    * number of different developers they have worked with (other people who have commited their authored diffs and authors whom they have commited as diffs) - both a count and who specifically
    * people who have worked on the same file
    * The following info is cumulative and static
    * total lines of code commited as a fraction of total lines of code
    * average change additions/deletions
    * number renames
    * number new files
    * number deletions

In [ ]:
auths = df3.sort_values(['author_id', 'timestamp']).groupby(['author_id', 'timestamp']).agg({
    'additions': np.sum, 'deletions': np.sum, 'net_change': np.mean,
    'code_complexity_median': np.mean, 'code_complexity_max': np.max,
    'filename_old': 'nunique', 'diff_id': 'count', 
    'is_rename': np.sum, 'is_deletion': np.sum, 'is_new': np.sum}).rename(columns={
        'additions': 'total_additions', 
        'deletions': 'total_deletions', 'net_change': 'average_net_change',
        'code_complexity_median':'mean_of_code_complexity_median', 
        'filename_old': 'num_unique_files_changed', 'diff_id': 'num_edit_locations',
        'is_rename': 'num_renames', 'is_deletion': 'num_deletions',
        'is_new': 'num_new_files'})
auths['net_code'] = auths['total_additions'] - auths['total_deletions']
auths_cum = auths.groupby('author_id').cumsum()

In [ ]:
auths2 = auths.merge(cum_repo_stats, left_index=True, right_index=True, how='left')
auths2['pct_total_additions'] = auths2['total_additions']/auths2['cum_total_lines_added']
auths2['pct_total_deleted'] = auths2['total_deletions']/auths2['cum_total_lines_deleted']
auths2['pct_total_lines'] = auths2['net_code']/auths2['cum_total_lines_code']


In [ ]:
auths_cum2 = auths_cum.merge(cum_repo_stats, left_index=True, right_index=True, how='left')
auths_cum2['pct_total_additions'] = auths_cum2['total_additions']/auths_cum2['cum_total_lines_added']
auths_cum2['pct_total_deleted'] = auths_cum2['total_deletions']/auths_cum2['cum_total_lines_deleted']
auths_cum2['pct_total_lines'] = auths_cum2['net_code']/auths_cum2['cum_total_lines_code']
auths_cum2_short = auths_cum2.drop(columns=[c for c in cum_repo_stats.columns], axis=1)
auths_cum2_short.columns = ['cum_{}'.format(c) for c in auths_cum2_short.columns ]

In [ ]:
auth_time = df3.sort_values(
    ['author_id', 'timestamp']).groupby(['author_id']).agg({'timestamp': [np.min, np.max]})
auth_time.columns = auth_time.columns.get_level_values(1)
auth_time2 = auth_time.rename(columns={
    'amin': 'first_author_engagement', 'amax':'last_author_engagement'})

In [ ]:
auths3 = auths2.merge(auth_time2, left_index=True, right_index=True, how='left')
auths4 = auths2.merge(auths_cum2_short, left_index=True, right_index=True, how='left')

In [ ]:
# collaborations stats
# for each author, I want to keep track of who commits their diffs
# who else works on the same files as them
author_comm_pairs = df3[['author_id', 'author_email', 'commiter_id', 'commiter_email', 'timestamp']].values

# filter for pairs that are different
collabs = list(filter(lambda x: x[0]!=x[2], author_comm_pairs))

# for each author, find number of commits
collab_df = pd.DataFrame(
    collabs, columns=['author_id', 'author_email', 'commiter_id', 'commiter_email', 'timestamp']).drop_duplicates()


collab2 = collab_df[['author_id', 'commiter_id']].sort_values('author_id').drop_duplicates().values
d = collections.defaultdict(list)
c = collections.defaultdict(list)
for auth_id, collab in collab2:
    d[auth_id].append(collab)
    c[collab].append(auth_id)
    

auths5 = auths4.merge(
    collab_df.groupby('author_id').agg(
        {'commiter_id': 'nunique'}).rename(columns={'commiter_id': 'num_different_commiters'}),
    left_index=True, right_index=True, how='left').fillna({'num_different_commiters': 0})
auths5 = auths5.sort_index()

In [ ]:
print(auths5.num_different_commiters.min())
print(auths5.num_different_commiters.max())
print(auths5.num_different_commiters.mean())


In [ ]:
# Look at commit behavior
comms = df3.sort_values(['commiter_id', 'timestamp']).groupby(['commiter_id', 'timestamp']).agg({
    'code_complexity_median': np.mean, 
    'filename_old': 'nunique', 'diff_id': 'count', 
    'is_rename': np.sum, 'is_deletion': np.sum, 'is_new': np.sum}).rename(columns={
        'code_complexity_median':'mean_of_code_complexity_median',  'code_complexity_max': np.max,
        'filename_old': 'num_unique_files_changed', 'diff_id': 'num_edit_locations',
        'is_rename': 'num_renames', 'is_deletion': 'num_deletions',
        'is_new': 'num_new_files'})
comms['net_code'] = auths['total_additions'] - auths['total_deletions']
comms_cum = comms.groupby('commiter_id').cumsum()

In [ ]:
# now look at all developers involved, plot number of authored comms, diffed comms, other collaborators, min auth, max auth, min comm, max comm
comm_time = df3.sort_values(
    ['commiter_id', 'timestamp']).groupby(['commiter_id']).agg({'timestamp': [np.min, np.max],
                                                               'commit_id': 'nunique'})
comm_time.columns = comm_time.columns.get_level_values(1)
comm_time2 = comm_time.rename(columns={
    'amin': 'first_commiter_engagement', 'amax':'last_commiter_engagement', 'nunique': 'num_commiter_commits'})

In [ ]:
auth_time = df3.sort_values(
    ['author_id', 'timestamp']).groupby(['author_id']).agg({
    'timestamp': [np.min, np.max], 
    'commit_id': 'nunique'})
auth_time.columns = auth_time.columns.get_level_values(1)
auth_time2 = auth_time.rename(columns={
    'amin': 'first_author_engagement', 'amax':'last_author_engagement', 'nunique': 'num_authored_commits'})

In [ ]:
# merge commiter and author time info
dev = auth_time2.merge(comm_time2, left_index=True, right_index=True, how='outer')

dev['people_who_committer_their_commits'] = dev.index.map(lambda x: d[x])

dev['people_who_authored_commits_they_commited'] = dev.index.map(lambda x: c[x])

auth_merge = author_df[['email', 'id', 'name']].rename(columns={'id': 'author_id'}).set_index('author_id')

dev2 = dev.merge(auth_merge, left_index=True, right_index=True, how='left')

dev2.index.name = 'author_id'
dev2 = dev2.sort_index()

devs2 = dev.merge(auths5, left_index=True, right_index=True, how='left')


In [ ]:
# need to add data set on group of people who only committed stuff

In [ ]:
print(devs2.shape)
print(auths5.shape)
print(dev.shape)
print(dev.index.unique().shape)
print(devs2.index.get_level_values(0).unique().shape)

In [ ]:
# export to CSV
if EXPORT_TO_CSV:
    auths5.to_csv(os.path.join(DROPBOX_DIR, DEV_CHANGES_FNAME))
    dev2.to_csv(os.path.join(DROPBOX_DIR, DEV_COLLAB_FNAME))

# File Level Stats


 1) time of first appearance, last appearance
 
 2) Number of developers who have worked on the file (over time)
 
 3) average size of code per change
 
 4) Other files changed with the file (adjacency matrix of changes)
 
 5) Person who has contributed most to the file
 
 6) Number of file renames
    
 7) when it appeared
 
 8) orginal author




In [ ]:
file_time = df3.sort_values(
    ['canonical_name', 'timestamp']).groupby(['canonical_name',]).agg({'timestamp': [np.min, np.max]})
file_time.columns = file_time.columns.get_level_values(1)
file_time2 = file_time.rename(columns={
    'amin': 'first_appearance', 'amax':'last_appearance'})

In [ ]:
files = df3.sort_values(['canonical_name', 'timestamp']).groupby(
    ['canonical_name', 'timestamp']).agg({
        'additions': np.sum, 'deletions': np.sum, 'net_change': np.sum,
        'diff_id': 'nunique',
        'is_rename': np.sum, 
        'is_deletion': lambda x: any(x)}).rename(columns={
        'additions': 'total_additions', 
        'deletions': 'total_deletions', 
        'net_change': 'total_net_change',
        'is_rename': 'num_renames',
        'is_deletion': 'is_deleted',
        'diff_id': 'num_changes'})

In [ ]:
# get first author, first commit, first commiter
first_dev = df3[
    ['canonical_name', 'timestamp', 'author_id', 'author_email', 'commiter_id', 'commiter_email']].sort_values([
        'canonical_name', 'timestamp']).drop_duplicates(subset=['canonical_name'], keep='first').set_index(
        'canonical_name').drop('timestamp', axis=1)

In [ ]:
cum_files = files.groupby('canonical_name').cumsum().rename(columns={'total_additions': 'cum_total_additions',
                                                        'total_deletions': 'cum_total_deletions',
                                                        'num_changes': 'cum_num_changes',
                                                        'num_renames': 'cum_num_renames',
                                                        'is_deleted': 'cum_is_deleted'})

In [ ]:
cum_files2 = cum_files.merge(first_dev, left_index=True, right_index=True, how='left')
print(first_dev.shape)
print(cum_files.shape)
print(cum_files2.shape)

In [ ]:
if EXPORT_TO_CSV:
    cum_files2.to_csv(os.path.join(DROPBOX_DIR, FILE_CUM_CHANGES_FNAME))

In [ ]:
# to get dev who contributed most, groupby filename, dev
# then you can also count distinct developers
# this gets total number of unique developers and commiters per file

# this will be merged into first_dev data set that tells us about the first author and commiter for each file
file_dev = df3.sort_values(
    ['canonical_name', 'author_id', 'timestamp']).groupby(['canonical_name']).agg({
    'author_id': 'nunique', 'commiter_id': 'nunique', 
    'additions': np.sum, 'deletions': np.sum,  'net_change': np.sum, 'code_complexity_max': np.max}
        ).rename(columns={
        'author_id': 'num_unique_devs','commiter_id': 'num_unique_commiters',
        'additions': 'total_additions', 'deletions': 'total_deletions',
        'net_change': 'total_code', 'code_complexity_max': 'max_complexity'})


In [ ]:
# for each file, also want to break out developers by percentage of code written
# groupby filename, developer, sum total lines of code (cum sum), then cum sum by dev and 
# then calculate a percentage


file_dev_lines = df3.sort_values(
    ['canonical_name', 'author_id', 'timestamp']).groupby(['canonical_name', 'author_id']).agg({
    'additions': np.sum, 'deletions': np.sum,  'net_change': np.sum, 'code_complexity_max': np.max,
    'timestamp': np.max})
file_dev_code_breakdown = file_dev_lines.merge(right=file_dev, left_index=True, right_index=True)
print(file_dev.shape)
print(file_dev_lines.shape)
print(file_dev_code_breakdown.shape)

In [ ]:
if EXPORT_TO_CSV:
    file_dev_code_breakdown.to_csv(os.path.join(DROPBOX_DIR, FILE_STATS_FNAME))

In [ ]:
# create a count of number of solo authored files for each developer
# also average age of the files they have worked on
# also average number of people on the files they have worked on
file_stats = file_dev_lines.reset_index('author_id').groupby(['canonical_name']).agg({'author_id': 'count', 'additions': np.sum, 'deletions': np.sum, 
                                               'code_complexity_max': np.max, 'timestamp': np.max}).rename(
    columns={'author_id': 'distinct_authors', 'additions': 'total_additions', 'deletions': 'total_deletions', 
            'code_complexity_max': 'file_level_code_complexity_max', 'timestamp': 'time_last_change'})


In [ ]:
file_stats

In [ ]:
file_stats2 = file_dev_lines.merge(file_stats, left_index=True, right_index=True, how='outer')

file_stats2['solo_authored'] = 0
file_stats2.loc[file_stats2.distinct_authors==1, 'solo_authored'] = 1

file_stats2['authored_max_complexity'] = 0
file_stats2.loc[((file_stats2.code_complexity_max==file_stats2.file_level_code_complexity_max) & (
    file_stats2.file_level_code_complexity_max > 0)), 'authored_max_complexity'] = 1

file_stats2['pct_additions'] = file_stats2['additions']*100.0/file_stats2['total_additions']
file_stats2['pct_deletions'] = file_stats2['deletions']*100.0/file_stats2['total_deletions']
file_stats2[['pct_additions', 'pct_deletions']] = file_stats2[['pct_additions', 'pct_deletions']].fillna(0)

file_stats2['more_than_50pct_additions'] = file_stats2.pct_additions.apply(lambda x: 1 if x > 50 else 0)
file_stats2['more_than_50pct_deletions'] = file_stats2.pct_deletions.apply(lambda x: 1 if x > 50 else 0)

file_stats2['last_change'] = file_stats2.apply(lambda x: 1 if x.timestamp==x.time_last_change else 0, axis=1)
file_stats2['file_age'] = file_stats2['time_last_change'].apply(lambda x: (pd.datetime.now() - x).days)

In [ ]:
if EXPORT_TO_CSV:
    file_stats2.to_csv(os.path.join(DROPBOX_DIR, DEV_CONTR_BY_FILE_FNAME))

In [ ]:
# same analysis, but cumulative over time
cum_code_by_dev = df3.sort_values(
    ['canonical_name', 'author_id', 'timestamp',]).groupby(
    ['canonical_name', 'author_id', 'timestamp']).agg(
    {'additions': np.sum, 'deletions': np.sum, 'net_change': np.sum}).groupby(
    ['canonical_name', 'author_id']).cumsum()


In [ ]:
if EXPORT_TO_CSV:
    cum_code_by_dev.to_csv(os.path.join(DROPBOX_DIR, FILE_DEV_CUM_STATS_FNAME))

In [ ]:
# get list of files that are usually changed together 
# grouped by commit id, get all canonical filename, create a matrix keeping count of times changed together
list_file_combos = []

gr = df3.sort_values(
    ['commit_id','timestamp', 'canonical_name'])[['commit_id', 'canonical_name']].groupby(['commit_id'])

for com_id, assoc_df in gr:
    list_file_combos.extend(list(itertools.combinations(assoc_df['canonical_name'].values, r=2)))
list_file_combos2 = list(filter(None, list_file_combos))   

In [ ]:
unique_filenames = set(funcy.flatten(list_file_combos2))

In [ ]:
arr1 = pd.Categorical(list(map(funcy.first, list_file_combos2)), categories=unique_filenames)
arr2 = pd.Categorical(list(map(funcy.second, list_file_combos2)), categories=unique_filenames)

In [ ]:
adjacency_matrix_filechanges = pd.crosstab(arr1, arr2)

In [ ]:
if EXPORT_TO_CSV:
    adjacency_matrix_filechanges.to_csv(os.path.join(DROPBOX_DIR, FILE_ADJ_MATRIX_FNAME))

In [ ]:
changes_per_file = df3.sort_values('canonical_filename').groupby('canonical_filename').size()
files = df3.groupby('canonical_filename').agg

In [ ]:
df3.columns

In [ ]:
file_time = df3.sort_values(['canonical_name', 'timestamp']).groupby(
    ['canonical_name', 'timestamp']).agg({
        'additions': np.sum, 'deletions': np.sum, 'net_change': np.mean,
        'code_complexity_median': np.mean, 
        'diff_id': 'count',
        'is_rename': lambda x: np.sum(list(map(int, x))), 
        'is_deletion': lambda x: any(x)}).rename(columns={
        'additions': 'total_additions', 
        'deletions': 'total_deletions', 
        'net_change': 'average_net_change',
        'is_rename': 'num_renames',
        'is_deletion': 'is_deleted',
        'code_complexity_median':'mean_of_code_complexity_median', 
        'diff_id': 'num_changes'})


In [ ]:
changes_per_file = df3.sort_values('canonical_filename').groupby('canonical_filename').size()
files = df3.groupby('canonical_filename').agg

In [ ]:
# plot number of file creations, deletions and renames
file_creations = df3[['timestamp', 'is_deletion', 'is_rename', 'is_new']].set_index(
    'timestamp').sort_index().cumsum().drop_duplicates()

In [ ]:
plt.figure(figsize=(20,15))
file_creations.plot()
plt.title('Cumulative File Deletions, New Files and Renames')
plt.savefig(os.path.join(FIGURE_DIR, 'cumulative_file_changes'.png', bbox_inches='tight')
plt.show()

In [ ]:
print(new_files.shape)

In [ ]:
# create a temporary column called filename_groupby that includes other filename when one value is dev/null
new_files = (df3['filename_old'] == '/dev/null')
df3['groupby_filename'] = df3['filename_old']
df3.loc[new_files, 'groupby_filename'] = df3[new_files]['filename_new']


In [ ]:
# df3[pd.isnull(df3['groupby_filename'])]
print(df3[df3['groupby_filename'] == ''].shape)
print(df3.shape)


In [ ]:
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
df3.groupby_filename.value_counts()[:30]

In [ ]:
plt.figure(figsize=(20,15))
df3.groupby_filename.value_counts()[:20].plot(kind='bar')
plt.title('Number of Diffs Per File')
plt.savefig('Diffs_per_file.png', bbox_inches='tight')

In [ ]:
plt.figure(figsize=(20,15))
df3.author_email.value_counts()[:20].plot(kind='bar')
plt.title('Number of Diffs Per Author')
plt.savefig('Diffs_per_author.png', bbox_inches='tight')

In [ ]:
frag = df3.groupby(
    'groupby_filename').agg({'additions': np.sum, 'deletions': np.sum, 'author_id': 'nunique'})
frag2 = frag.rename(columns={
    'additions': 'total_additions', 'deletions': 'total_deletions', 'author_id':'unique_contributors'}).sort_values(
    'unique_contributors', ascending=False)

In [ ]:
frag2[:30]

In [ ]:
fig, ax1 = plt.subplots(figsize=(20, 15))

color = 'tab:red'
# ax1.set_xlabel('filename')
ax1.set_ylabel('Lines of Code', color=color)
lns1 = ax1.plot(frag2[:30].index, frag2[:30].total_additions, 'r-', label='total_additions')
lns2 = ax1.plot(frag2[:30].index, frag2[:30].total_deletions, 'g-', label='total_deletions')
ax1.tick_params(axis='y', labelcolor=color)
plt.xticks(rotation='vertical')

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
ax2.legend()
color = 'tab:blue'
ax2.set_ylabel('number unique contributors', color=color)  # we already handled the x-label with ax1
lns3 = ax2.plot(frag2[:30].index, frag2[:30].unique_contributors, color=color)
ax2.tick_params(axis='y', labelcolor=color)

lns = lns1+lns2+lns3
labs = [l.get_label() for l in lns]
ax1.legend(lns, labs, loc=0)

plt.title('Fragmentation')
plt.show()
fig.savefig('frag.png', bbox_inches='tight')


In [ ]:
df.head()

In [ ]:
# for each filename, groupby filename
df2 = df.groupby('filename_new')

In [ ]:
new_info = []
for fname_new, fname_old, commit_id, _ in filename_tups:
    commit_info = session.query(Commit).filter(Commit.id==commit_id).first()
    new_info.append((
        a, b, commit_info.timestamp,
            commit_info.subject, commit_info.commit_body, commit_info.author_id,
                commit_info.commiter_id, ))
    

In [ ]:
session.query(Commit).filter(Commit.id==1529).first().author

In [ ]:
session.query(Diff).first().__dict__


In [ ]:
filename_tups

# for each of the files changed, I need to know when a change occured, who changed it, how many insertions, deletions did they do, what commit was it part of, what was the commit 
# for each of the new filenames I need to know lines of code per day added
# then, I need to show that by developer
# for code fragementation, I need a running average of total lines of code and the number contributed by different developers
# then for code churn I need to calculate number of changes per file 
# then i want to dvivide dev by first itneraction with repo
# for dependence I either need to look at who builds on one, or I need to look at who create repos and does first commits into a project (feature)
# for each commit, number of files changed can be a proxy
# then, for each dev, I want to be able to check who are the group of people they work on code with or change the same file with
# for complexity, 

## checks
# why do some files have no changes?
# what can I do to add location?
# how can i track renames or deletions
# what about the boilerplate is throwing this off?